In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Basic Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from collections import OrderedDict

# Dataset class
From https://www.kaggle.com/pinocookie/pytorch-dataset-and-dataloader

load image as ndarray type (Height * Width * Channels)
be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
in this example, i don't use ToTensor() method of torchvision.transforms
so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)

In [ ]:
class DatasetMNIST(torch.utils.data.Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28, 1))
        label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            #print("---")
            image = self.transform(image)
            
        return image, label

Setup key parameters

In [ ]:
lRate   = 1.0
lGamma  = 0.7
lEpochs = 1#5
log_interval = 100

# Net

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential( OrderedDict([
            ('conv1', nn.Conv2d( 1,  8, 3, 2) ),
            ('relu1', nn.ReLU() ),
            ('conv2', nn.Conv2d( 8,  16, 3, 2) ),
            ('relu2', nn.ReLU() ),
            ('conv3', nn.Conv2d( 16,  32, 3, 2) ),
            ('relu3', nn.ReLU() ),
            #('maxpl', nn.MaxPool2d( 2 ) ),
            ('flatt', nn.Flatten() ),
            ('fc3'  , nn.Linear( 128, 10) ),
            #('relu3', nn.ReLU() ),
            #('fc4'  , nn.Linear( 64, 10) ),
            #('relu4', nn.ReLU() ),
        ]))
        self.firstRun= 1;
    
    def forward(self, x):
        for layer in self.net:
            if self.firstRun == 1:
                print( x.shape )
            x = layer(x)
        output = F.log_softmax(x, dim=1)
        self.firstRun= 0;
        return output

# Loaders

In [ ]:
device = torch.device("cpu")
the_kwargs = {'num_workers': 1, 'shuffle': True, }
the_transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,)),
                       ])
train_loader = torch.utils.data.DataLoader(
        DatasetMNIST('../input/mnist-in-csv/mnist_train.csv', the_transform),
        batch_size=64, **the_kwargs)


test_set = DatasetMNIST('../input/mnist-in-csv/mnist_test.csv', the_transform)

test_loader = torch.utils.data.DataLoader(
        test_set, batch_size=1000, **the_kwargs)

# Prepare Learning

In [ ]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lRate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=lGamma )

# Main Scripts

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Learning

In [ ]:
for epoch in range(1, lEpochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

# Sample Mistakes

In [ ]:
(smplStd,ansStd) = next(iter(test_loader))
lst  = F.nll_loss( model(smplStd), ansStd, reduction='none' )
bads = smplStd[ torch.nonzero( lst>1 ) ]
bads = bads.reshape( (-1, 1, 28, 28) )
bedA = model(bads.reshape(-1,1,28,28))
abc = torch.argmax(bedA, dim=1)

# Show Some Mistakes

In [ ]:
for j in range (2):
    for i in range (8):
      if bads.size()[0] > i +j*8 :
        plt.subplot(2,8, i +j*8+1)
        plt.imshow( bads[i +j*8].reshape(28,28), cmap="gray_r")
        bedaE = enumerate( bedA[i +j*8] )
        print( ansStd[ torch.nonzero( lst>1 ) ][i +j*8], abc[i +j*8] ) 

# Result Export
from https://www.kaggle.com/furkanuysl/digit-recognizer-cnn-99

In [ ]:
test= test_set.data
total_samples = test.shape[0]
print(test.shape)
#test.head()
test = test.values.reshape(-1,(28*28 +1)) [:,1:].reshape(-1,1,28,28)
model.eval()
y_pred_test = model( torch.Tensor(test))
prediction = np.argmax(y_pred_test.detach().numpy(), axis = 1)

# create submission DataFrame
submission = pd.DataFrame({'ImageId' : range(1, total_samples+1), 'Label' : list(prediction)})
submission.head(10)
print (submission.shape)
submission.to_csv("submission.csv",index=False)